In [84]:
import json
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten

import os
os.getcwd()
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' # sonst raucht der Kernel auf dem Macbook Pro ab

In [85]:
# Zeichnet Boards 
### XXX TBC: TEST AUF FEHLER EINBAUEN

"""
def board_to_string (X,y, width = 3):
    output = "\n\nX = Stein von Spieler am Zug, 0 = Stein von anderem Spieler, * = Gute Züge\n\›"
    layer_char = ["X","0","A","B","C","D","E"] # eigentlich kommt nur X (spieler am zug), 0 (spieler am zug) und * als empfohlener Zug vor 
    for i, data_elem in enumerate (X):
        output += ("\ndata_elem: " + str(i) + "\n\n")
        for j in range (pixels):
            t_set = False
            for k, layer in enumerate (data_elem):
                if layer [j] == 1:
                    output += layer_char [k]
                    t_set = True
            if y [i][j] == 1:
                output += "*"
                t_set = True                    
            if not t_set:
                output += "."                
            if (j + 1) % width == 0:
                output += "\n"      
        output += "\n"                  
    return output

"""
def drawData(X,y):
    # print("X",X)
    bu = "┌───┬───┬───┐"
    bm = "├───┼───┼───┤"
    bd = "└───┴───┴───┘"
    for i, board in enumerate(X):
        # print("i:",i)
        # print("board:",board)
        for row in range(7):      
                if row == 0:
                    print(bu)
                elif row == 6:
                    print(bd)
                elif row % 2 == 0:
                    print(bm)
                else:
                    for j in range(3):
                        # mark = str(turn["board"][int(j+5/2*(row-1))])
                        k = int(j+3/2*(row-1))
                        #print("k:",k)
                        mark = str(int(board[0][k]) + 2*int(board[1][k]) -int(y[i][k]))
                        #print("mark:",mark)
                        #print("y:",-int(y[i][k]))
                        if mark == "0":
                            mark = " "
                        elif mark == "1":
                            mark = "X"
                        elif mark == "2":
                            mark = "O"
                        elif mark == "-1":
                            mark = "*"
                        print("│ " + mark + " ",end="")
                    print("│")
        print("")



def drawValues(prediction,X_test,y_test):
    bu = "┌───┬───┬───┐"
    bm = "├───┼───┼───┤"
    bd = "└───┴───┴───┘"
    for a in range(len(prediction)):
        for row in range(7):
            for i in range (2):

            # print("i:",i)
            # print("board:",board)

                if row == 0:
                    print(bu,end=" ")
                elif row == 6:
                    print(bd,end=" ")
                elif row % 2 == 0:
                    print(bm,end=" ")
                else:
                    for j in range(3):
                        # mark = str(turn["board"][int(j+5/2*(row-1))])
                        k = int(j+3/2*(row-1)) #int(j+3/2*(row-1))
                        #print("k:",k)
                        #print("c",c)
                        if i == 0:
                            mark = "{:.1f}".format(prediction[a][k])
                            if mark == "-0.0":
                                mark = "0.0"
                                
                        else:
                            mark = " " + str(int(real[a][k])) + " " #str(int(board[0][k]) + 2*int(board[1][k]) -int(y[i][k]))
                        #print("mark:",mark)
                        print("│" + mark + "",end="")
                    print("│ ", end="")
            print("")
        
        
# nimmt eine stellung in der form array[0,1,2,0,0,2,1,0,0], z.B. .
# Stellung 0:
#    xx.
#    .00
#    x..
# und gibt ein array [8,9] aus: Die Stellung selbst, drei Drehungen (90°,180°,270°) und die 4 Spiegelungen von Stellung 0-3 an der X oder Y Achse (egal)
# Stellung 1:
#   x.x 
#   .0x
#   .0.
#   
# Spiegelung
#   .0.
#   .0x
#   x.x
#
# Die Drehungen und Spiegelungen sind topologisch aus der Sicht des Spiels identisch, wenn die Folgezüge, dann auch jeweils gedreht und gespiegelt werden
#
#

def transform(input_array):
    corners = [[1,3,9,7], [7,9,3,1]]
    edges = [[2,6,8,4], [8,6,2,4]]
    output_arrays = []
    for variant in range(2):
        for turn in range(4):
            new_array = [0,0,0,0,0,0,0,0,0]
            for position in range(1,10):
                for i in range(4):
                    if position == corners[variant][i]:
                        new_array[corners[0][(i+turn)%4]-1] = input_array[position-1]
                    elif position == edges[variant][i]:
                        new_array[edges[0][(i+turn)%4]-1] = input_array[position-1]
                    elif position == 5:
                        new_array[4] = input_array[position-1]
            output_arrays.append(new_array)
    return output_arrays

In [86]:
datafile = open('../01_data/Data_9F_Basics.json',)
# returns JSON object as a dictionary
dataset = json.load(datafile)
datasize = len(dataset)

layers = 2     # players
pixels = 3 * 3 # cells


# print (dataset)
# print (dataset[1]["field"][4])
# print (dataset[2]["good_moves"][2])

# cleanly suck it up

X = np.zeros([datasize, layers, pixels]) # e.g. 30 data samples with 2 (p)layers and a board of 3 x 3 
y = np.zeros([datasize, pixels]) # maximum possible moves = pixels

for i, data_elem in enumerate (dataset): # data_elem = 1 learning sample 
    # print ("*** data_elem", i,"-", data_elem)
    
    """# who's turn is it? Die Steine des ziehenden Spieler kommen in input layer 0 des nnetzes, die des anderen in 1
    t_num, t_counts = np.unique(data_elem["board"], return_counts=True) # gibt etwas wie [0,1,2] [6,2,1] zurück => 2 Steine von P1, 1 von P2 
    counts = dict(zip(t_num, t_counts))
    if not 1 in counts: counts[1] = 0
    if not 2 in counts: counts[2] = 0
    # print ("counts", counts)        
    if counts[1] > counts[2]: # wenn P1 einen stein mehr auf dem Feld hat als P2, ist P2 dran, sonst P1
        turn = 2
    else:         
        turn = 1
    # print ("turn",turn)
    """
    z = 0
    for p, pixel in enumerate(data_elem["board"]):
        if pixel == 0:
            z += 1
    turn = (pixels-z)%2
    
    for p, pixel in enumerate (data_elem["board"]):
        # print (j, pixel)
        # The layer of the drawing player
        if pixel == turn+1:
            X[i,0,p] = 1.0
        elif pixel > 0:
            X[i,1,p] = 1.0
    for g, good_move in enumerate (data_elem["good_moves"]):
        y[i,good_move-1] = 1.0
        
     
                
# print ("X:\n", X) 
# print ("...")
# print ("y:\n", y)
            

drawData(X,y)

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │ X │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬─

In [87]:
# enriched_with_all 1+7 transformations
X_enriched = np.zeros([datasize*8, layers, pixels]) # e.g. 30 data samples with 2 (p)layers and a board of 3 x 3 
y_enriched = np.zeros([datasize*8, pixels]) # maximum possible moves = pixels


"""
for i, data_elem in enumerate (X): # data_elem = 1 learning sample 
        # print ("# i", i)
        for j, layer in enumerate (data_elem):
            # print ("## j", j)
            for k, transformation_layer in enumerate (transform (layer)):
                # print (k, transformation_layer)
                X_enriched [i*8+k,j] = transformation_layer
            ### Papusch, hier noch schalter rein, dass es nur jedes zweite mal aufgerufen wird
            if (j == 0):
                for k, transformation_layer in enumerate (transform (y [i])):
                    y_enriched [i*8+k] = transformation_layer
"""


for i, x_elem in enumerate(X): 
    for l, layer in enumerate(x_elem):
        for t, trans in enumerate(transform(layer)):
            X_enriched[i*8+t,l] = trans
            
for i, y_elem in enumerate(y):
    for t, trans in enumerate(transform(y_elem)):
            y_enriched[i*8+t] = trans

# print(X_enriched)
# print(y_enriched)
# print (board_to_string (X_enriched, y_enriched))

drawData(X_enriched,y_enriched)


┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ O │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬─

│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│   │ O │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│   │ * │ X │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│ X │ * │   │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│ X │ * │   │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│   │ * │ X │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ X │ * │
├───┼───┼───┤
│ O │ O │   │
├───┼───┼───┤
│   │   │ 

│ X │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ X │ * │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│ * │   │ O │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ * │ X │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│ O │   │ * │
├───┼───┼───┤
│   │ O │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ * │ X │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│   │   │ * │
├───┼───┼───┤
│ O │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │   │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│ X │ * │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│ * │   │   │
├───┼───┼───┤
│ X │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ O │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ X │ * │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│ * │   │   │
├───┼───┼───┤
│ X │   │ 

│ * │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │   │   │
├───┼───┼───┤
│ O │ X │ O │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ * │ O │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ * │   │   │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│ O │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ * │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │   │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ O │ O │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ X │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ O │   │ 

├───┼───┼───┤
│ O │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │   │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ O │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼───┤
│   │ O │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │   │
├───┼───┼───┤
│ O │ * │   │
├───┼───┼───┤
│ O │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼───┤
│ O │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ X │   │
├───┼───┼───┤
│ O │ * │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ O │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │   │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│   │ X │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ O │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼─

In [88]:

# print(datasize)
# bringt die X und y in eine zufällige (aber gleiche) Reihenfolge

shuffler = np.random.permutation(datasize*8)
X_enriched = X_enriched[shuffler]
y_enriched = y_enriched[shuffler]

# print (board_to_string (X_enriched, y_enriched))
drawData(X_enriched,y_enriched)

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│ O │ * │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│ X │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ X │   │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │   │ O │
├───┼───┼───┤
│ X │ X │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ * │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │ X │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬─

│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │ O │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │   │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ X │ X │ * │
├───┼───┼───┤
│   │ O │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ O │ * │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ X │ O │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ X │   │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │   │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼───┤
│ O │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│ X │ X │ * │
├───┼───┼───┤
│   │   │ 

│ X │ O │   │
├───┼───┼───┤
│ * │   │   │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ X │ X │
├───┼───┼───┤
│   │ O │   │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│ O │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│ * │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ X │
├───┼───┼───┤
│ O │ * │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │   │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│   │ O │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │ O │ * │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│ * │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │   │ O │
├───┼───┼───┤
│ X │ * │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │ 

│   │   │ * │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│ X │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│ O │ * │   │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ X │ * │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ * │   │ O │
├───┼───┼───┤
│ X │   │ O │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ * │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │ X │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│   │ O │ X │
├───┼───┼───┤
│   │ * │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ O │
├───┼───┼───┤
│ * │ X │ X │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │   │   │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│ O │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ 

└───┴───┴───┘

┌───┬───┬───┐
│ X │ X │ * │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │   │
├───┼───┼───┤
│ X │ * │ X │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │   │
├───┼───┼───┤
│ * │ X │ X │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │   │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ * │ X │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ X │ * │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│ O │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ * │ X │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ X │   │
├───┼───┼───┤
│ * │   │   │
├───┼───┼───┤
│ O │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴─

In [89]:
testsize = 128; # number of test samples, always the last in the file
trainsize = datasize*8 - testsize

X_train = X_enriched [:datasize*8 - testsize]
y_train = y_enriched [:datasize*8 - testsize]

X_test = X_enriched [-testsize:]
y_test = y_enriched [-testsize:]

print ("\n\n\nTRAINDATA\n")
# print (board_to_string (X_train, y_train))
drawData(X_train,y_train)

print ("\n\n\nTESTDATA\n")
# print (board_to_string (X_test, y_test))
drawData(X_test,y_test)

# print ("X_train:\n", X_train) 
# print ("y_train:\n", y_train) 
# print ("###")
# print ("X_test:\n", X_test) 
# print ("y_test:\n", y_test) 
# print ("###")
# print ("###")




TRAINDATA

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│ O │ * │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│ X │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ X │   │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│ O │   │ O │
├───┼───┼───┤
│ X │ X │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ * │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │ X │
├───┼───┼───┤
│   │   │ X │
└───┴───┴──

│ O │   │ O │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│ * │ X │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ X │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ O │
├───┼───┼───┤
│   │ X │ * │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ * │ X │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│   │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│ O │   │ X │
├───┼───┼───┤
│ O │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ O │
├───┼───┼───┤
│   │   │ * │
├───┼───┼───┤
│   │ X │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│ X │ * │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ * │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│ X │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ O │
├───┼───┼───┤
│   │ O │ X │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ * │ 

├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│ X │ X │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│ X │   │ O │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │   │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│ O │ X │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ * │
├───┼───┼───┤
│   │ X │ O │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ * │
├───┼───┼───┤
│ O │   │ X │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ O │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│ O │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│ * │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ * │   │   │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│ O │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ O │ * │
├───┼───┼───┤
│ X │   │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ O │   │
├───┼───┼─

│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│   │ * │ O │
├───┼───┼───┤
│   │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │   │
├───┼───┼───┤
│ O │   │ X │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│ O │ O │ * │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ O │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│   │ X │ * │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ O │
├───┼───┼───┤
│   │   │ * │
├───┼───┼───┤
│   │   │ O │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ O │ * │ O │
├───┼───┼───┤
│   │   │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ O │
├───┼───┼───┤
│ O │ * │   │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│ X │   │ O │
├───┼───┼───┤
│ * │ O │   │
├───┼───┼───┤
│ X │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│ * │ X │ X │
├───┼───┼───┤
│   │ O │ 

├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ X │ O │   │
├───┼───┼───┤
│ O │ X │   │
├───┼───┼───┤
│   │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ X │
├───┼───┼───┤
│   │ O │ X │
├───┼───┼───┤
│ O │   │ * │
└───┴───┴───┘

┌───┬───┬───┐
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
├───┼───┼───┤
│ * │ * │ * │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │ O │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│   │ X │ O │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ * │
├───┼───┼───┤
│ O │   │ X │
├───┼───┼───┤
│ O │   │ X │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │   │
├───┼───┼───┤
│   │   │ O │
├───┼───┼───┤
│ * │ X │ X │
└───┴───┴───┘

┌───┬───┬───┐
│ O │   │ X │
├───┼───┼───┤
│ O │   │   │
├───┼───┼───┤
│ * │   │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ O │ X │
├───┼───┼───┤
│   │ * │   │
├───┼───┼───┤
│ X │ O │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │ X │   │
├───┼───┼───┤
│ O │ * │ O │
├───┼───┼───┤
│   │ X │   │
└───┴───┴───┘

┌───┬───┬───┐
│   │   │ O │
├───┼───┼─

In [91]:
# Anmerkung Rocky: Mal die Anzahl der Neuronen variieren

print("Testsize :",testsize)
print("Trainsize:",trainsize)
print("\n")

model = Sequential()
model.add(Dense(pixels, input_shape=(layers,pixels), activation='relu',name="input"))
model.add(Flatten(name="flatten"))
model.add(Dense(90, activation='relu', name="h1"))
model.add(Dense(90, activation='relu', name="h2"))
model.add(Dense(90, activation='relu', name="h3"))
# model.add(Dense(80, activation='relu', name="h3"))
# model.add(Dense(80, activation='relu', name="h4"))
#model.add(Dense(60, activation='relu', name="h4"))
#model.add(Dense(40, activation='relu', name="h4"))
#model.add(Dense(pixels*layers, activation='relu', name="h4"))
#model.add(Dense(pixels*layers, activation='relu', name="h5"))
model.add(Dense(pixels, name="output"))
model.summary ()

# compile the keras model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')
# model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse'])

# fit the keras model on the dataset
model.fit (X_train, y_train, epochs=500, batch_size=30)

# evaluate the keras model
mae, mse = model.evaluate(X_train, y_train)
print('Mean squared error train: %.4f' % (mse))
mae, mse = model.evaluate(X_test, y_test)
print('Mean squared error test: %.4f' % (mse))

# make class predictions with the model
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)


# print (len(y_train))

#print ("\n\n### TRAIN ###")
#for i, elem in enumerate (predictions_train):
#    print(i, np.around(predictions_train[i], 2), y_train[i])

print ("\n\n\nTEST\n")
"""for i, elem in enumerate (predictions_test):
    print(i, np.around(predictions_test[i], 2), y_test[i])"""

drawValues(predictions_test,X_test,y_test)
    

         



testsize: 128
trainsize: 472
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 2, 9)              90        
_________________________________________________________________
flatten (Flatten)            (None, 18)                0         
_________________________________________________________________
h1 (Dense)                   (None, 90)                1710      
_________________________________________________________________
h2 (Dense)                   (None, 90)                8190      
_________________________________________________________________
h3 (Dense)                   (None, 90)                8190      
_________________________________________________________________
output (Dense)               (None, 9)                 819       
Total params: 18,999
Trainable params: 18,999
Non-trainable params: 0
_____________________

Epoch 72/500
472/472 [==============================] - 0s 207us/step - loss: 0.0842 - mse: 0.0825
Epoch 73/500
472/472 [==============================] - 0s 205us/step - loss: 0.0846 - mse: 0.0825
Epoch 74/500
472/472 [==============================] - 0s 235us/step - loss: 0.0841 - mse: 0.0824
Epoch 75/500
472/472 [==============================] - 0s 213us/step - loss: 0.0844 - mse: 0.0824
Epoch 76/500
472/472 [==============================] - 0s 222us/step - loss: 0.0843 - mse: 0.0823
Epoch 77/500
472/472 [==============================] - 0s 213us/step - loss: 0.0839 - mse: 0.0822
Epoch 78/500
472/472 [==============================] - 0s 207us/step - loss: 0.0838 - mse: 0.0821
Epoch 79/500
472/472 [==============================] - 0s 233us/step - loss: 0.0837 - mse: 0.0819
Epoch 80/500
472/472 [==============================] - 0s 216us/step - loss: 0.0837 - mse: 0.0817
Epoch 81/500
472/472 [==============================] - 0s 241us/step - loss: 0.0836 - mse: 0.0816
Epoch 82/5

472/472 [==============================] - 0s 582us/step - loss: 0.0788 - mse: 0.0773
Epoch 155/500
472/472 [==============================] - 0s 602us/step - loss: 0.0788 - mse: 0.0772
Epoch 156/500
472/472 [==============================] - 0s 579us/step - loss: 0.0785 - mse: 0.0772
Epoch 157/500
472/472 [==============================] - 0s 349us/step - loss: 0.0785 - mse: 0.0771
Epoch 158/500
472/472 [==============================] - 0s 250us/step - loss: 0.0786 - mse: 0.0770
Epoch 159/500
472/472 [==============================] - 0s 212us/step - loss: 0.0785 - mse: 0.0768
Epoch 160/500
472/472 [==============================] - 0s 218us/step - loss: 0.0784 - mse: 0.0767
Epoch 161/500
472/472 [==============================] - 0s 207us/step - loss: 0.0782 - mse: 0.0765
Epoch 162/500
472/472 [==============================] - 0s 209us/step - loss: 0.0782 - mse: 0.0764
Epoch 163/500
472/472 [==============================] - 0s 214us/step - loss: 0.0778 - mse: 0.0763
Epoch 164/500


472/472 [==============================] - 0s 346us/step - loss: 0.0767 - mse: 0.0756
Epoch 237/500
472/472 [==============================] - 0s 376us/step - loss: 0.0768 - mse: 0.0756
Epoch 238/500
472/472 [==============================] - 0s 312us/step - loss: 0.0770 - mse: 0.0756
Epoch 239/500
472/472 [==============================] - 0s 233us/step - loss: 0.0768 - mse: 0.0756
Epoch 240/500
472/472 [==============================] - 0s 265us/step - loss: 0.0766 - mse: 0.0756
Epoch 241/500
472/472 [==============================] - 0s 283us/step - loss: 0.0770 - mse: 0.0756
Epoch 242/500
472/472 [==============================] - 0s 255us/step - loss: 0.0769 - mse: 0.0756
Epoch 243/500
472/472 [==============================] - 0s 262us/step - loss: 0.0770 - mse: 0.0756
Epoch 244/500
472/472 [==============================] - 0s 233us/step - loss: 0.0766 - mse: 0.0756
Epoch 245/500
472/472 [==============================] - 0s 228us/step - loss: 0.0765 - mse: 0.0756
Epoch 246/500


472/472 [==============================] - 0s 242us/step - loss: 0.0767 - mse: 0.0755
Epoch 319/500
472/472 [==============================] - 0s 235us/step - loss: 0.0766 - mse: 0.0755
Epoch 320/500
472/472 [==============================] - 0s 240us/step - loss: 0.0765 - mse: 0.0754
Epoch 321/500
472/472 [==============================] - 0s 237us/step - loss: 0.0765 - mse: 0.0754
Epoch 322/500
472/472 [==============================] - 0s 227us/step - loss: 0.0767 - mse: 0.0755
Epoch 323/500
472/472 [==============================] - 0s 281us/step - loss: 0.0764 - mse: 0.0755
Epoch 324/500
472/472 [==============================] - 0s 319us/step - loss: 0.0764 - mse: 0.0755
Epoch 325/500
472/472 [==============================] - 0s 212us/step - loss: 0.0764 - mse: 0.0755
Epoch 326/500
472/472 [==============================] - 0s 218us/step - loss: 0.0767 - mse: 0.0755
Epoch 327/500
472/472 [==============================] - 0s 215us/step - loss: 0.0766 - mse: 0.0754
Epoch 328/500


472/472 [==============================] - 0s 277us/step - loss: 0.0765 - mse: 0.0754
Epoch 401/500
472/472 [==============================] - 0s 224us/step - loss: 0.0766 - mse: 0.0754
Epoch 402/500
472/472 [==============================] - 0s 231us/step - loss: 0.0765 - mse: 0.0754
Epoch 403/500
472/472 [==============================] - 0s 205us/step - loss: 0.0765 - mse: 0.0754
Epoch 404/500
472/472 [==============================] - 0s 225us/step - loss: 0.0763 - mse: 0.0754
Epoch 405/500
472/472 [==============================] - 0s 282us/step - loss: 0.0766 - mse: 0.0754
Epoch 406/500
472/472 [==============================] - 0s 254us/step - loss: 0.0765 - mse: 0.0754
Epoch 407/500
472/472 [==============================] - 0s 238us/step - loss: 0.0764 - mse: 0.0754
Epoch 408/500
472/472 [==============================] - 0s 225us/step - loss: 0.0764 - mse: 0.0754
Epoch 409/500
472/472 [==============================] - 0s 230us/step - loss: 0.0765 - mse: 0.0754
Epoch 410/500


472/472 [==============================] - 0s 208us/step - loss: 0.0755 - mse: 0.0746
Epoch 483/500
472/472 [==============================] - 0s 201us/step - loss: 0.0755 - mse: 0.0746
Epoch 484/500
472/472 [==============================] - 0s 214us/step - loss: 0.0755 - mse: 0.0746
Epoch 485/500
472/472 [==============================] - 0s 222us/step - loss: 0.0755 - mse: 0.0746
Epoch 486/500
472/472 [==============================] - 0s 286us/step - loss: 0.0756 - mse: 0.0746
Epoch 487/500
472/472 [==============================] - 0s 243us/step - loss: 0.0756 - mse: 0.0746
Epoch 488/500
472/472 [==============================] - 0s 216us/step - loss: 0.0757 - mse: 0.0747
Epoch 489/500
472/472 [==============================] - 0s 206us/step - loss: 0.0757 - mse: 0.0746
Epoch 490/500
472/472 [==============================] - 0s 226us/step - loss: 0.0756 - mse: 0.0746
Epoch 491/500
472/472 [==============================] - 0s 230us/step - loss: 0.0758 - mse: 0.0746
Epoch 492/500


TypeError: drawValues() takes 2 positional arguments but 3 were given

[[1. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0.]]


In [36]:
# Erstes Array ist AI
a_1 = np.array([[[0., 0., 1., 1., 1., 0., 0., 1., 0.],[1., 1., 0., 0., 0., 0., 1., 0., 1.]],])

 
print (np.around (model.predict(a_1),2))

[[-0.   -0.   -0.    0.01  0.2   0.51 -0.    0.   -0.  ]]


In [ ]:
"""
def drawDatal(X,y):
    # print("X",X)
    bu = "┌───┬───┬───┐"
    bm = "├───┼───┼───┤"
    bd = "└───┴───┴───┘"
    for i, board in enumerate(X):
        # print("i:",i)
        # print("board:",board)
        for row in range(7):      
                if row == 0:
                    print(bu)
                elif row == 6:
                    print(bd)
                elif row % 2 == 0:
                    print(bm)
                else:
                    for j in range(3):
                        # mark = str(turn["board"][int(j+5/2*(row-1))])
                        k = int(j+3/2*(row-1))
                        #print("k:",k)
                        mark = str(int(board[0][k]) + 2*int(board[1][k]) -int(y[i][k]))
                        #print("mark:",mark)
                        #print("y:",-int(y[i][k]))
                        if mark == "0":
                            mark = " "
                        elif mark == "1":
                            mark = "X"
                        elif mark == "2":
                            mark = "O"
                        elif mark == "-1":
                            mark = "*"
                        print("│ " + mark + " ",end="")
                    print("│")
        print("")



def drawValues(prediction,X_test,y_test):
    bu = "┌───┬───┬───┐"
    bm = "├───┼───┼───┤"
    bd = "└───┴───┴───┘"
    for a in range(len(prediction)):
        for row in range(7):
            for i in range (2):

            # print("i:",i)
            # print("board:",board)

                if row == 0:
                    print(bu,end=" ")
                elif row == 6:
                    print(bd,end=" ")
                elif row % 2 == 0:
                    print(bm,end=" ")
                else:
                    for j in range(3):
                        # mark = str(turn["board"][int(j+5/2*(row-1))])
                        k = int(j+3/2*(row-1)) #int(j+3/2*(row-1))
                        #print("k:",k)
                        #print("c",c)
                        if i == 0:
                            mark = "{:.1f}".format(prediction[a][k])
                            if mark == "-0.0":
                                mark = "0.0"
                                
                        else:
                            mark = " " + str(int(real[a][k])) + " " #str(int(board[0][k]) + 2*int(board[1][k]) -int(y[i][k]))
                        #print("mark:",mark)
                        print("│" + mark + "",end="")
                    print("│ ",end="")
            print("")

"""

In [128]:
"""# # #

# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# load the dataset
dataset = loadtxt('../01_data/pima-indians-diabetes.data.csv', delimiter=',')
# dataset = loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

print (X[0],X[1])

# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary ()

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y, epochs=20, batch_size=10)

# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

#
#

# make class predictions with the model
predictions = model.predict_classes(X)
# summarize the first 5 cases
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))"""

[  6.    148.     72.     35.      0.     33.6     0.627  50.   ] [ 1.    85.    66.    29.     0.    26.6    0.351 31.   ]
Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_174 (Dense)            (None, 12)                108       
_________________________________________________________________
dense_175 (Dense)            (None, 8)                 104       
_________________________________________________________________
dense_176 (Dense)            (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
768/768 [==============================] - 1s 1ms/step - loss: 21.0317 - accuracy: 0.3477
Epoch 2/20
768/768 [==============================] - 0s 188us/step - loss: 7.7701 - accuracy: 0.3971
Epoch 3/20
768/768 [==============================] - 0s 1